## Week 1 : Large Language Models
```
- Generative Artificial Intelligence (Fall semester 2023)
- Professor: Muhammad Fahim
- Teaching Assistant: Ahmad Taha
```
<hr>

## Contents
```
1. Transformers (Implementing a transformer)
2. Self-Attention
3. Multi-headed attention
4. Positional Encoding

```

<hr>


# Transformers

* [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) -- Original paper on attention

![](http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png)


In [1]:
import torch
from torch import nn
import torch.optim as optim
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### Transformer Encoder with Pytorch

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=32)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

In [3]:
encoder_layer

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
  )
  (linear1): Linear(in_features=512, out_features=2048, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=2048, out_features=512, bias=True)
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

## Encoder

The encoder contains a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. <br>
**The main goal is to efficiently encode the data**

![](http://jalammar.github.io/images/t/encoder_with_tensors.png)

## Self-Attention

**Keep in mind : The main goal is to encode the data in a much more efficient way** In other words is to create meaningful embeddings<br>
- As the model processes each word (each position in the input sequence), self attention allows it to look at other positions in the input sequence for clues that can help lead to a better encoding for this word.


**How does Self-Attention work?**

Steps:
1. For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.
  - What are the **`Query`** vector, a **`Key`** vector, and a **`Value`** vector? : They’re abstractions that are useful for calculating attention... They are a breakdown of the word embeddings
2. Calculating self-attention score from **`Query`** **`Key`** vector.
3. Divide the scores by 8 (This leads to having more stable gradients)
4. Pass the result through a softmax operation (softmax score determines how much each word will be expressed at this position)
5. Multiply each value vector by the softmax score
6. Sum up the weighted value vectors

### Step 1

For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.

![](http://jalammar.github.io/images/t/transformer_self_attention_vectors.png)

In [4]:
# simple sequence = I am here today
simple_sequence_embedding = torch.rand((4, 512))

# Create weight matrices
Wq = torch.normal(0,0.5, (512, 7))
Wk = torch.normal(0,0.1, (512, 7))
Wv = torch.normal(0,0.2, (512, 7))

# Create key, query and value for each word in the senetence
queries = simple_sequence_embedding.mm(Wq) # self.q(embedding[0])
keys = simple_sequence_embedding.mm(Wk)
values = simple_sequence_embedding.mm(Wv)

In [5]:
values

tensor([[ 2.5932, -3.2220, -0.3001,  3.4596,  2.8656, -0.8417, -4.0470],
        [ 0.7683, -1.2544,  2.7111,  0.9582,  5.6401, -1.1286, -5.0858],
        [ 2.8754, -0.2318,  1.8981,  2.3130,  2.9996,  0.0587, -1.1642],
        [-0.1914, -1.6522,  2.2397,  2.1431,  5.2016, -1.6490, -3.9122]])

In [6]:
simple_sequence_embedding

tensor([[0.0192, 0.3369, 0.4398,  ..., 0.8528, 0.3716, 0.2348],
        [0.2730, 0.4278, 0.6589,  ..., 0.7106, 0.2968, 0.0330],
        [0.7017, 0.8688, 0.9488,  ..., 0.0027, 0.2373, 0.6707],
        [0.6083, 0.3034, 0.7345,  ..., 0.2399, 0.2169, 0.5537]])

## Step 2

Calculating self-attention score from **`Query`** and **`Key`** vector

In [7]:
scores = torch.mm(queries, keys.T)
scores

tensor([[23.4544, 25.5804, 13.9753, 26.0646],
        [20.3942, 26.7025, 21.0241, 18.7530],
        [21.9645, 18.2919, 11.9148, 21.8382],
        [32.4823,  7.8042, 15.8128, 18.4249]])

## Step 3
Divide the scores by 8 (This leads to having more stable gradients)

In [8]:
scores = scores / 8
scores

tensor([[2.9318, 3.1976, 1.7469, 3.2581],
        [2.5493, 3.3378, 2.6280, 2.3441],
        [2.7456, 2.2865, 1.4893, 2.7298],
        [4.0603, 0.9755, 1.9766, 2.3031]])

## Step 4

Pass the result through a softmax operation

In [9]:
scores = torch.nn.functional.softmax(scores, dim=1)
scores

tensor([[0.2503, 0.3264, 0.0765, 0.3468],
        [0.1962, 0.4317, 0.2123, 0.1598],
        [0.3447, 0.2178, 0.0982, 0.3393],
        [0.7447, 0.0341, 0.0927, 0.1285]])

## Step 5 & 6

* Multiply each value vector by the softmax score
* Sum up the weighted value vectors



In [10]:
scores.shape, values.shape

(torch.Size([4, 4]), torch.Size([4, 7]))

In [11]:
z = scores @ values
z

tensor([[ 1.0534, -1.8065,  1.7319,  2.0988,  4.5916, -1.1464, -4.1188],
        [ 1.4203, -1.4870,  1.8724,  1.9259,  4.4651, -0.9034, -3.8619],
        [ 1.2786, -1.9673,  1.4333,  2.3555,  4.2757, -1.0897, -3.9446],
        [ 2.1994, -2.6760,  0.3326,  3.0990,  3.2727, -0.8717, -3.7979]])

# Multi-headed attention

**GOAL**:
1. Expand the model’s ability to focus on different positions
2. Provide the attention layer multiple “representation subspaces”

**Attention with $N$ just means repeating self attention algorithm $N$ times and joining the results**


![](https://data-science-blog.com/wp-content/uploads/2022/01/mha_img_original.png)

**Multi-headed attention steps:**
1. Same as self-attention calculation, just n different times with different weight matrices
2. Condense the $N$ z metrices down into a single matrix by concatinating the matrices then multiply them by an additional weights matrix `WO`

Now the output z metrix is fed to the FFNN

In [ ]:
from torch import Tensor
import torch.nn.functional as f


def scaled_dot_product_attention(query, key, value):
  temp = query.bmm(key.transpose(1, 2))
  scale = query.size(-1) ** 0.5
  softmax = f.softmax(temp / scale, dim=-1)
  return softmax.bmm(value)

## Now lets make attention head

In [12]:
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, dim_in, dim_q, dim_k):
        super().__init__()
        # Fill in the missing parts of the constructor to initialize the query, key, and value linear transformations
        # dim_in is the input dimension, dim_q and dim_k are the output dimensions for the queries and keys/values respectively
        # Example: dim_in 512, dim_q = dim_k 64 in the paper
        self.query_linear = nn.Linear(dim_in, dim_q)
        self.key_linear = nn.Linear(dim_in, dim_k)
        self.value_linear = nn.Linear(dim_in, dim_k)
        

    def forward(self, query, key, value):
        # Implement the forward pass by calling scaled_dot_product_attention function
        # You need to transform the query, key, and value using the linear transformations defined in __init__
        # Fill in with the correct method calls
        query_transformed = self.query_linear(query)
        key_transformed = self.key_linear(key)
        value_transformed = self.value_linear(value)
        
        return scaled_dot_product_attention(query_transformed, key_transformed, value_transformed)

## Multi Head Attention

In [15]:
from torch import Tensor
class MultiHeadToAttention(nn.Module):
    def __init__(self, number_of_heads, dim_in, dim_q, dim_k):
        super().__init__()
        # Initialize heads as multi-AttentionHead instances
        # Initialize linear to combine the outputs of all heads into a single output
        self.heads = nn.ModuleList(
            [AttentionHead(dim_in, dim_q, dim_k) for _ in range(number_of_heads)]
        )
        self.linear = nn.Linear(number_of_heads * dim_k, dim_in)

    def forward(self, query: Tensor, key: Tensor, value: Tensor):
        # Concatenate outputs from all heads and apply the final linear transformation
        head_outputs = [head(query, key, value) for head in self.heads]
        concatenated_outputs = torch.cat(head_outputs, dim=-1)
        final_output = self.linear(concatenated_outputs)
        return final_output

## Positional Encoding

A way to account for the order of the words in the input sequence. A transformer adds a vector to each input embedding which helps it determine the position of each word. <br>
**Goal** : preserving information about the order of tokens  

Positional Encoding they can either be learned or fixed a priori.

Proposed approach from original paper : describe a simple scheme for fixed positional encodings based on sine and cosine functions

![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*C3a9RL6-SFC6fW8NGpJg5A.png)

In [16]:
def position_encoding(seq_len, dim_model, device):
    # Define the position tensor 'pos' with dimensions appropriate for sequence length
    pos = torch.arange(seq_len, dtype=torch.float, device=device).unsqueeze(1)

    # Define the dimension tensor 'dim' suitable for the model dimensions
    dim = torch.arange(dim_model, dtype=torch.float, device=device).unsqueeze(0)

    # Calculate the phase using the position and dimension tensors
    phase = pos / (10000 ** (2 * (dim // 2) / dim_model))

    # Return the sinusoidal position encoding
    pos_encoding = torch.zeros((seq_len, dim_model), device=device)
    pos_encoding[:, 0::2] = torch.sin(phase[:, 0::2])
    pos_encoding[:, 1::2] = torch.cos(phase[:, 1::2])
    return  pos_encoding # Complete this line to select sin or cos based on even/odd index

## Encoder Feed Forward

In [26]:
def feed_forward(dim_input = 512, dim_feedforward = 2048):
  return nn.Sequential(nn.Linear(dim_input, dim_feedforward),
                       nn.ReLU(),
                       nn.Linear(dim_feedforward, dim_input)
                       )

## Encoder Residual

From the original paper the author implementation

In [27]:
class Residual(nn.Module):
  def __init__(self, sublayer, dimension, dropout = 0.1):
    super().__init__()
    self.sublayer = sublayer
    self.norm = nn.LayerNorm(dimension)
    self.dropout = nn.Dropout(dropout)

  def forward(self, *tensors):
    return self.norm(tensors[0] + self.dropout(self.sublayer(*tensors)))

## Putting all together on encoder side

![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)

## Putting the Encoder layer together

In [17]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, dim_model=512, num_heads=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Define dimensions for query and key based on model dimension and number of heads
        dim_q = dim_k = dim_model // num_heads

        # Initialize the MultiHeadAttention component with a residual connection and dropout
        self.attention = Residual(
            MultiHeadToAttention(num_heads, dim_model, dim_q, dim_k), dim_model, dropout
        )

        # Initialize the feedforward component with a residual connection and dropout
        self.feed_forward = Residual(
            feed_forward(dim_model, dim_feedforward), dim_model, dropout
        )

    def forward(self, src):
        # Apply the attention mechanism to the source data
        src = self.attention(src, src, src)

        # Apply the feedforward network to the output of the attention mechanism
        src = self.feed_forward(src)
        return src

## Putting together transfomer Encoder part

In [18]:
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers=12, dim_model=512, num_heads=4, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Initialize a list of TransformerEncoderLayer instances
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(dim_model, num_heads, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])
        self.dim_model = dim_model

    def forward(self, src):
        # Retrieve the sequence length and dimension from the source tensor
        seq_len, dimension = src.size(1), src.size(2)

        # Add position encoding to the source tensor
        src += position_encoding(seq_len, self.dim_model, src.device)

        # Process each layer in the transformer encoder

        for layer in self.layers:
            src = layer(src)

        return src

# The Decoder Side

The encoder start by processing the input sequence. The output of the top encoder is then transformed into a set of attention vectors K and V. These are to be used by each decoder.


![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)

## Decoder layer

The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the Keys and Values matrix from the output of the encoder stack.

**Task**: implement the decoder layer

In [19]:
class TransformerDecoderLayer(nn.Module):
    def __init__(
        self,
        dim_model: int = 512,
        num_heads: int = 6,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        # init dim_q and dim_k as in the encoder
        dim_q = dim_k = dim_model // num_heads
        # Initialize the first self-attention layer with a residual connection
        self.attention_1 = Residual(
            MultiHeadToAttention(num_heads, dim_model, dim_q, dim_k), dim_model, dropout
        )

        # Initialize the second attention layer for interaction with the encoder output
        self.attention_2 = Residual(
            MultiHeadToAttention(num_heads, dim_model, dim_q, dim_k), dim_model, dropout
        )

        # Initialize the feed-forward network
        self.feed_forward = Residual(
            feed_forward(dim_model, dim_feedforward), dim_model, dropout
        )

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Self-attention mechanism
        tgt = self.attention_1(tgt, tgt, tgt)

        # Cross-attention mechanism where the decoder attends to the encoder's output
        tgt = self.attention_2(tgt, memory, memory)

        # Pass through the feed-forward network
        tgt = self.feed_forward(tgt)
        return tgt

## Full Transfomer Decoder

**Task**: implement the transfomer decoder part class

In [20]:
import torch.nn.functional as F
class TransformerDecoder(nn.Module):
    def __init__(
        self,
        num_layers: int = 6,
        dim_model: int = 512,
        num_heads: int = 8,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        # Initialize laters from TransformerDecoderLayer instances
        self.layers = nn.ModuleList([
            TransformerDecoderLayer(dim_model, num_heads, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])
        # Define the output linear transformation
        self.linear = nn.Linear(dim_model, dim_model)

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Calculate sequence length and dimension from the target tensor
        seq_len, dimension = tgt.size(1), tgt.size(2)

        # Add position encoding to the target tensor
        tgt += position_encoding(seq_len, dimension, tgt.device)

        # Process each layer in the transformer decoder
        # Code here
        for layer in self.layers:
            tgt = layer(tgt, memory)

        output = self.linear(tgt)

        # Apply a softmax to the output of the final linear layer
        output = F.softmax(output, dim=-1)
        return output

## Full Transfomer model

**Task**:
1. Assembly a full transfomer (Encoder + Decoder)
2. Implement the Transfomer training loop
3. Using dataset of your choice train the transformer just for one epoch


In [21]:
class Transformer(nn.Module):
    def __init__(self, num_encoder_layers, num_decoder_layers, dim_model, num_heads, dim_feedforward, input_vocab_size, target_vocab_size, dropout=0.1):
        super().__init__()
        self.encoder = TransformerEncoder(num_layers=num_encoder_layers, dim_model=dim_model, num_heads=num_heads, dim_feedforward=dim_feedforward, dropout=dropout)
        self.decoder = TransformerDecoder(num_layers=num_decoder_layers, dim_model=dim_model, num_heads=num_heads, dim_feedforward=dim_feedforward, dropout=dropout)
        self.input_embedding = nn.Embedding(input_vocab_size, dim_model)
        self.target_embedding = nn.Embedding(target_vocab_size, dim_model)
        self.linear = nn.Linear(dim_model, target_vocab_size)

    def forward(self, src, tgt):
        src_embedded = self.input_embedding(src)
        tgt_embedded = self.target_embedding(tgt)

        encoder_output = self.encoder(src_embedded)
        decoder_output = self.decoder(tgt_embedded, encoder_output)

        output = self.linear(decoder_output)

        return output

In [22]:
def calculate_loss(predictions, targets, pad_index):
    loss_fn = nn.CrossEntropyLoss(ignore_index=pad_index)
    loss = loss_fn(predictions.view(-1, predictions.size(-1)), targets.view(-1))
    return loss

In [29]:
def train_epoch(model, dataloader, optimizer, pad_index):
    model.train()
    total_loss = 0
    for src, tgt in dataloader:
        optimizer.zero_grad()
        predictions = model(src, tgt[:, :-1] if tgt.dim() > 1 else tgt)
        loss = calculate_loss(predictions, tgt[:, 1:] if tgt.dim() > 1 else tgt, pad_index)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

In [24]:
def train_transformer(model, train_loader, optimizer, pad_index, num_epochs=1):
    for epoch in range(num_epochs):
        loss = train_epoch(model, train_loader, optimizer, pad_index)
        print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")

In [32]:
if __name__ == "__main__":
    # Параметры модели
    num_encoder_layers = 6
    num_decoder_layers = 6
    dim_model = 512
    num_heads = 8
    dim_feedforward = 2048
    input_vocab_size = 10000
    target_vocab_size = 10000
    dropout = 0.1

    # Создание модели
    model = Transformer(
        num_encoder_layers=num_encoder_layers,
        num_decoder_layers=num_decoder_layers,
        dim_model=dim_model,
        num_heads=num_heads,
        dim_feedforward=dim_feedforward,
        input_vocab_size=input_vocab_size,
        target_vocab_size=target_vocab_size,
        dropout=dropout
    )

    # Пример оптимизатора и загрузчика данных (необходимо заменить на реальные данные)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_loader = [(torch.randint(1, input_vocab_size, (10,)), torch.randint(1, target_vocab_size, (10,))) for _ in range(100)]

    # Обучение модели
    train_transformer(model, train_loader, optimizer, pad_index=0)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)